In [1]:
import os
os.chdir('../')

In [2]:
from lib.readfile import ReadFile as rf
from lib.handle_null_values import handle_null_values as null
from lib.datetime_formatting import DatetimeFormatting as dfmt
from lib.split_time_variable import split_time_variable as split_t
from lib.correct_variable_types import correct_variable_types as vartype

In [4]:
"""Read the file"""

r= rf()
df= r.read(address="lib/data/2009.csv")

In [5]:
"""Remove column with null values"""

n= null()
variables= n.delete_var_with_null_more_than(df)

In [6]:
"""drop unwanted variables for now"""

df2= df.drop(*variables)

In [7]:
"""select important variables""" 

important_variables=['year_built', 'bedroom_count', 'bathroom_count','lot_size', 'area', 'bike_score', 'city_name', 'community_name', 'covered_parking', 'created_date', 'exterior', 'energy_source','fireplaces_count', 'floor_covering', 'garage','heating_cooling','latitude','longitude', 'property_type', 'roof', 'school_district',  'sewer', 'sold_date', 'sold_price','unfinished_sq_foot', 'topography', 'transit_score', 'was_flip', 'year_built', 'zip_code', 'county' ]

    
important_variables= list(set(important_variables))
df3= df2.select(*important_variables)



In [8]:
"""concatenate all files"""

files= ["09"]
for f_no in range(10, 20):
    files.append(f_no)    
    
    df4= r.read(address="lib/data/20"+str(10)+".csv")
    df4= df4.select(*important_variables)
    df3 = df3.union(df4)


In [9]:
"""filter one county from the dataset"""

df5= df3.filter( df3["county"].isin(["King"]))

In [10]:
"""correct time format"""

f= split_t()

#Assuming time variable is in timestamp type
df6= f.run(df5)


In [12]:
"""Correct variable types"""

v= vartype()

df7= v.run(df6)


In [13]:
p=df6.limit(500).toPandas()

numeric_columns= []
columns2= df6.columns # only take string variables

In [14]:
len(list(set(columns2)))

37

In [15]:
p['year_built']
# There is a duplication on the 

0      1959
1      1989
2      1995
3      1984
4      2006
       ... 
495    2006
496    1968
497    1975
498    2005
499    1975
Name: year_built, Length: 500, dtype: object

In [ ]:
for c in columns2:
    counter= 0
    e=[]
    float_values= []
    print(c)
    for i in range(500):
        try:
            v= float(p[c][i])
            counter= counter+1
            float_values.append(v)
        except:
            print(str(p[c][i]))
            e.append(str(p[c][i]))

    if len(e)==0:
        numeric_columns.append(c)  
    elif len(set(float_values))<5: 
        if len(set(e))<=1 and len(set(float_values))!=0:
            numeric_columns.append(c)  
    elif len(float_values)/500*100 > 95:
            numeric_columns.append(c)

In [16]:
var=find_numeric_variables_saved_as_string(df6)

NameError: name 'find_numeric_variables_saved_as_string' is not defined

In [ ]:
"""treat duplications"""
df6.select('year_built').show()

In [44]:
"""treat null values using deep learning"""

# add an ID in the dataset
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark import SparkContext
from pyspark.sql import SparkSession
from math import *


spark_context = SparkContext.getOrCreate()
spark = SparkSession(spark_context)

windowSpec = W.orderBy(df7.columns[0])
df7= df7.withColumn("id", F.row_number().over(windowSpec))
df8= df7.createOrReplaceTempView("customer")

#use this ID to filter the data and get results.
base_query = "SELECT * FROM customer where "
total_rows= df7.count()
each_page= 500
total_pages= ceil(total_rows/each_page)

for p in range(total_pages):
    query= base_query + "id > " + str(each_page*p) + " and id< " + str(each_page*(p+1))
    df_spark= spark.sql(query)

    df_pandas= df_spark.toPandas()
    
    
    





#save the model for future use.

In [45]:
df_pandas

,address,unfinished_sq_foot,longitude,covered_parking,topography,sewer,energy_source,school_district,bike_score,sold_price,...,transit_score,latitude,sold_date_year,sold_date_month,sold_date_day,sold_datedayofweek,sold_datehour,sold_datemin,sold_datesec,id
0,10024 20th Ave NE,180.0,-122.307503,1.0,"Brush, Fruit Trees, Garden Space, Partial Slop...",G,"Electric, Natural Gas",Seattle,0.0,580000.0,...,0.0,47.701688,2009,8,21,6,0,0,0,1001
1,10024 Densmore Ave N,0.0,-122.337478,2.0,Fruit Trees,G,"Electric, Wood",Seattle,0.0,375000.0,...,0.0,47.702098,2010,6,2,4,0,0,0,1002
2,10024 Densmore Ave N,0.0,-122.337478,2.0,Fruit Trees,G,"Electric, Wood",Seattle,0.0,375000.0,...,0.0,47.702098,2010,6,2,4,0,0,0,1003
3,10024 Densmore Ave N,0.0,-122.337478,2.0,Fruit Trees,G,"Electric, Wood",Seattle,0.0,375000.0,...,0.0,47.702098,2010,6,2,4,0,0,0,1004
4,10024 Densmore Ave N,0.0,-122.337478,2.0,Fruit Trees,G,"Electric, Wood",Seattle,0.0,375000.0,...,0.0,47.702098,2010,6,2,4,0,0,0,1005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,10040 Ravenna Ave NE,0.0,-122.299257,2.0,"Fruit Trees, Level",G,"Electric, Wood",Seattle,0.0,432500.0,...,0.0,47.702334,2010,7,21,4,0,0,0,1495
495,10040 Ravenna Ave NE,0.0,-122.299257,2.0,"Fruit Trees, Level",G,"Electric, Wood",Seattle,0.0,432500.0,...,0.0,47.702334,2010,7,21,4,0,0,0,1496
496,10040 Ravenna Ave NE,0.0,-122.299257,2.0,"Fruit Trees, Level",G,"Electric, Wood",Seattle,0.0,432500.0,...,0.0,47.702334,2010,7,21,4,0,0,0,1497
497,10040 Ravenna Ave NE,0.0,-122.299257,2.0,"Fruit Trees, Level",G,"Electric, Wood",Seattle,0.0,432500.0,...,0.0,47.702334,2010,7,21,4,0,0,0,1498


In [ ]:
for p in range(total_pages):
    query= base_query + "id > " + str(each_page*p) + " and id< " + str(each_page*(p+1))
    df_spark= spark.sql(query)

    df_pandas= df_spark.toPandas()
    

In [47]:
# train the deep learning model and put that on the training
categorical_variables= []
numerical_variables=[]
for dt in df7.dtypes:
    if dt[1]=="string":
        categorical_variables.append(dt[0])
    else:
        numerical_variables.append(dt[0])
        
        

In [77]:
df_train2= df7.toPandas()

In [78]:
df_train2

,address,unfinished_sq_foot,longitude,covered_parking,topography,sewer,energy_source,school_district,bike_score,sold_price,...,transit_score,latitude,sold_date_year,sold_date_month,sold_date_day,sold_datedayofweek,sold_datehour,sold_datemin,sold_datesec,id
0,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,1
1,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,2
2,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,3
3,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,4
4,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209022,9981 Avondale Rd NE,0.0,-122.093338,0.0,None,None,Natural Gas,Lake Washington,0.0,270000.0,...,0.0,47.688652,2010,3,8,2,0,0,0,209023
209023,9981 Avondale Rd NE,0.0,-122.093338,0.0,None,None,Natural Gas,Lake Washington,0.0,270000.0,...,0.0,47.688652,2010,3,8,2,0,0,0,209024
209024,9981 Avondale Rd NE,0.0,-122.093338,0.0,None,None,Natural Gas,Lake Washington,0.0,270000.0,...,0.0,47.688652,2010,3,8,2,0,0,0,209025
209025,9981 Avondale Rd NE,0.0,-122.093338,0.0,None,None,Natural Gas,Lake Washington,0.0,270000.0,...,0.0,47.688652,2010,3,8,2,0,0,0,209026


In [65]:
import datawig
#Initialize a SimpleImputer model
df_train = df7.limit(10000).toPandas()
    


In [81]:
count= 209027
train_df= df_train2[:int(count*.9)]
test_df= df_train2[int(count*.9):]

In [83]:
train_df

,address,unfinished_sq_foot,longitude,covered_parking,topography,sewer,energy_source,school_district,bike_score,sold_price,...,transit_score,latitude,sold_date_year,sold_date_month,sold_date_day,sold_datedayofweek,sold_datehour,sold_datemin,sold_datesec,id
0,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,1
1,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,2
2,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,3
3,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,4
4,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188119,769 31 Ave,310.0,-122.292615,0.0,Garden Space,G,Natural Gas,Seattle,0.0,473200.0,...,0.0,47.609016,2011,6,1,4,0,0,0,188120
188120,769 31 Ave,310.0,-122.292615,0.0,Garden Space,G,Natural Gas,Seattle,0.0,473200.0,...,0.0,47.609016,2011,6,1,4,0,0,0,188121
188121,769 31 Ave,310.0,-122.292615,0.0,Garden Space,G,Natural Gas,Seattle,0.0,473200.0,...,0.0,47.609016,2011,6,1,4,0,0,0,188122
188122,769 Hayes St,0.0,-122.341472,0.0,None,None,Electric,Seattle,0.0,255500.0,...,0.0,47.633954,2009,12,29,3,0,0,0,188123


In [ ]:
results= {}
models={}

for c in categorical_variables: 
    var= categorical_variables.copy()
    var.remove(c)
    imputer = datawig.SimpleImputer(
        input_columns=var, # column(s) containing information about the column we want to impute
        output_column=c, # the column we'd like to impute values for
        output_path = 'lib/imputer_models' # stores model data and metrics
        )
    imputer.fit(train_df=train_df)
    models[c]= imputer
    #time to test the model with entire file.  
    
#     results[c] = imputer.predict(df_pandas)
    

    

2019-10-08 14:56:51,254 [INFO]  CategoricalEncoder for column address                                found only 95 occurrences of value 124 Warren Ave N
2019-10-08 14:56:51,256 [INFO]  CategoricalEncoder for column address                                found only 95 occurrences of value 10610 NE 9th Place 
2019-10-08 14:56:51,258 [INFO]  CategoricalEncoder for column address                                found only 94 occurrences of value 2222 152nd Ave NE
2019-10-08 14:56:51,260 [INFO]  CategoricalEncoder for column address                                found only 94 occurrences of value 1400 Hubbell Place 
2019-10-08 14:56:51,263 [INFO]  CategoricalEncoder for column address                                found only 82 occurrences of value 1420 Terry Ave 
2019-10-08 14:56:51,264 [INFO]  CategoricalEncoder for column address                                found only 80 occurrences of value 6970 California Ave SW
2019-10-08 14:56:51,266 [INFO]  CategoricalEncoder for column address 

In [72]:
results=imputer.predict(df_pandas)

In [74]:
results.to_csv("lib/clean.csv")

In [ ]:
df7.select('address').show

In [ ]:
"""encode categorical variables."""

In [ ]:
"""split data in to training and testing dataset"""

In [ ]:
p=df.limit(500).toPandas()

In [ ]:
p["ml_number"][1]